In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
import env
import acquire
import explore
import prepare

In [2]:
#Sets options to show more information
pd.options.display.max_columns = None
pd.options.display.width = 100
pd.options.display.max_colwidth = None
pd.options.display.max_rows = 200

In [3]:
df = acquire.wrangle_zillow()

In [4]:
df_sorted = df.sort_values(['parcelid','transactiondate'], ascending=False)

In [5]:
df_sorted = df_sorted[~df_sorted['parcelid'].duplicated()]

In [6]:
#Fill with zeros
cols_to_fill = ['basementsqft', 'decktypeid', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
                'hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid2','pooltypeid7',
                'pooltypeid10','fireplaceflag', 'taxdelinquencyflag', 'airconditioningdesc'] 

In [7]:
df_sorted[cols_to_fill] = df_sorted[cols_to_fill].fillna(0)

In [8]:
df_sorted = df_sorted[df_sorted['latitude'].notna()]

In [9]:
df_sorted.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,id.1,logerror,transactiondate,propertylandusedesc,storydesc,typeconstructiondesc
16654,167689317,NaN,NaN,269.0,NaN,NaN,NaN,NaN,2535331,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34045062.0,-118020570.0,NaN,0.0,0.0,0.0,0.0,0.0,010D,EMR1B*,6.037434e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,318400.0,437387.0,2016.0,118987.0,5947.25,0,NaN,NaN,0,NaN,NaN,NaN,16654,-0.008789,2017-03-14,Planned Unit Development,NaN,NaN
7892,167688532,NaN,NaN,266.0,2.0,NaN,NaN,1.0,2758757,0.0,3.0,3.0,4.0,3.0,0.0,NaN,1661.0,1661.0,NaN,NaN,NaN,NaN,6037.0,0.0,3.0,0.0,0.0,0.0,34108983.0,-118262402.0,NaN,0.0,0.0,0.0,0.0,0.0,010C,LARD2,6.037187e+07,NaN,3101.0,NaN,NaN,0.0,NaN,1.0,NaN,NaN,2016.0,NaN,0.0,NaN,147921.0,2016.0,147921.0,1902.75,0,NaN,NaN,Central,NaN,NaN,Central,7892,0.006706,2017-02-03,Condominium,NaN,NaN
41688,167687839,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1372384,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34048223.0,-118520239.0,NaN,0.0,0.0,0.0,0.0,0.0,0100,LAR1,6.037263e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1842678.0,2016.0,1842678.0,22045.81,0,NaN,NaN,0,NaN,NaN,NaN,41688,0.038797,2017-05-31,Single Family Residential,NaN,NaN
14557,167687739,NaN,NaN,266.0,NaN,NaN,NaN,NaN,2863262,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34041716.0,-118455310.0,NaN,0.0,0.0,0.0,0.0,0.0,010C,LAR3,6.037268e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5451600.0,6440197.0,2016.0,988597.0,77045.13,0,NaN,NaN,0,NaN,NaN,NaN,14557,0.360020,2017-03-03,Condominium,NaN,NaN
13467,167686999,NaN,NaN,261.0,NaN,NaN,NaN,NaN,775695,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6037.0,0.0,NaN,0.0,0.0,0.0,34424104.0,-118468083.0,NaN,0.0,0.0,0.0,0.0,0.0,0100,SCRM,6.037920e+07,NaN,3101.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,26405.0,2016.0,26405.0,988.48,0,NaN,NaN,0,NaN,NaN,NaN,13467,-0.068632,2017-02-28,Single Family Residential,NaN,NaN


In [10]:
df_sorted = prepare.handle_missing_values(df_sorted)

Size of Correlation	Interpretation

|.90 to 1.00| (−.90 to −1.00)|	Very high positive (negative) correlation|

|.70 to .90| (−.70 to −.90)|	High positive (negative) correlation|

|.50 to .70| (−.50 to −.70)|	Moderate positive (negative) correlation|

|.30 to .50| (−.30 to −.50)|	Low positive (negative) correlation|

|.00 to .30| (.00 to −.30)|	negligible correlation|

In [11]:
df_sorted.shape

(77381, 50)

In [12]:

def compare_num_cols(df, alpha=0.05):

    """
    Maybe create a function that automatically seperates continuous from discrete columns.
    """

    list_of_cols = df.select_dtypes(include=[int, float]).columns
    
    for item in list_of_cols:
        target_col = item
        metrics = []
        for col in list_of_cols:
            result = stats.anderson(df[col])
            #Checks skew to pick a test
            if result.statistic < result.critical_values[2]:
                corr, p_value = stats.pearsonr(df[target_col],
                                               df[col])
                test_type = '(P)'
            else:
                # I'm unsure how this handles columns with null values in it.
                corr, p_value = stats.spearmanr(df[target_col],
                                                df[col], nan_policy='omit')
                test_type = '(S)'

            #Answer logic
            if p_value < alpha:
                test_result = 'relationship'
            else:
                test_result = 'independent'

            temp_metrics = {"Column":f'{col} {test_type}',
                            "Correlation": corr,
                            "P Value": p_value,
                            "Test Result": test_result}
            metrics.append(temp_metrics)

        distro_df = pd.DataFrame(metrics)              
        distro_df = distro_df.set_index('Column')


        #Remove highly correlated features 
        distro_df = distro_df[distro_df['Correlation'].abs() < 0.7]

        #Column of absolute values
        distro_df['Corr_abs'] = distro_df['Correlation'].abs()

        #How do I get these items but in a while dataframe
        distro_df = distro_df.sort_values('Corr_abs', ascending=False).head()

        #Plotting the relationship with the target variable (and stats test result)
        my_range=range(1,len(distro_df.index) + 1)
        hue_colors = {'relationship': 'green', 'independent':'red'}

        #This is the plotting section
        plt.figure(figsize=(6,1.25))
        plt.axvline(0, c='tomato', alpha=.6)

        plt.hlines(y=my_range, xmin=-1, xmax=1, color='grey', alpha=0.4)
        sns.scatterplot(data=distro_df, x="Correlation",
                        y=my_range, hue="Test Result", palette=hue_colors,
                        style="Test Result", hue_order=hue_colors)
        plt.legend(title="Stats test result")

        # Add title and axis names
        plt.yticks(my_range, distro_df.index)
        plt.title(f'Statistics tests of {target_col}', loc='center')
        plt.xlabel('Neg Correlation            No Correlation            Pos Correlation')
        plt.ylabel('Feature')

        #Saves plot when it has a name and uncommented
        #plt.savefig(f'{train.name}.png')

In [13]:
#compare_num_cols(df_sorted)

In [14]:
df_sorted.corr()

,parcelid,propertylandusetypeid,heatingorsystemtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id.1,logerror
parcelid,1.000000,0.033473,0.108641,0.000794,0.021531,0.002311,0.006501,-0.047052,0.014576,0.107438,0.026302,0.036296,0.461816,0.269516,-0.026856,0.382869,0.292591,0.052278,-0.167139,-0.013450,-0.087221,-0.062702,0.123226,0.028397,0.043630,-0.077055,0.461452,-0.019592,-0.359738,0.050139,0.339518,0.012738,0.141876,0.019763,0.031760,0.012742,NaN,0.004126,0.001851,-0.034123,0.104321,0.000241,0.015830
propertylandusetypeid,0.033473,1.000000,-0.091481,-0.000657,-0.002031,-0.044926,-0.268979,0.307971,-0.122148,0.017272,-0.224383,-0.243267,0.055055,-0.019834,-0.136426,0.047214,-0.083172,-0.021052,-0.020218,-0.005145,0.145325,0.076161,-0.016215,-0.009651,-0.018746,0.083624,0.055276,-0.017476,-0.072551,0.006532,-0.018558,-0.261489,0.348386,0.016700,-0.004394,-0.035544,NaN,-0.045126,-0.040405,0.005907,0.007018,0.009920,-0.005600
heatingorsystemtypeid,0.108641,-0.091481,1.000000,0.000574,NaN,-0.316655,-0.163225,-0.627714,-0.316749,NaN,-0.237264,-0.245465,0.594007,0.046596,-0.331235,0.400858,0.065712,0.030117,-0.338324,0.267623,-0.141530,-0.200285,NaN,0.028598,0.016256,-0.201614,0.589816,0.040297,-0.594007,-0.002098,0.539732,0.144557,-0.303087,0.039853,-0.178138,-0.110142,NaN,-0.064664,-0.118303,-0.032453,0.010071,-0.015837,-0.006500
id,0.000794,-0.000657,0.000574,1.000000,0.006361,0.002994,0.001866,0.005646,0.003110,-0.000984,0.004918,0.003624,-0.007922,-0.002827,0.004179,-0.002242,-0.003200,-0.000502,-0.002667,0.006324,-0.003875,0.001803,-0.005379,-0.003859,0.001947,0.001242,-0.007922,0.003797,0.001793,-0.002161,-0.005729,-0.002310,-0.000254,0.002306,0.003728,0.001694,NaN,0.000472,0.001281,-0.024659,-0.003875,-0.000374,0.002045
basementsqft,0.021531,-0.002031,NaN,0.006361,1.000000,0.026344,0.011559,NaN,0.026579,0.080932,0.014336,0.015466,0.053756,0.069802,0.024406,0.033624,0.034926,-0.002554,0.017302,-0.037974,0.000074,0.005054,0.044695,-0.001393,-0.002128,0.005830,0.053901,-0.000206,-0.010588,0.001059,0.042240,NaN,-0.000559,-0.000847,0.006418,0.008309,NaN,0.008016,0.006487,NaN,0.007177,-0.004107,0.004518
bathroomcnt,0.002311,-0.044926,-0.316655,0.002994,0.026344,1.000000,0.635158,0.524698,1.000000,0.049081,0.772845,0.803969,0.015986,0.106287,0.984481,0.147166,0.197003,0.166056,0.026538,-0.023665,0.002488,0.217909,0.086608,0.053779,0.162620,0.175283,0.016223,-0.036719,-0.024730,0.018167,0.037715,0.062864,0.373455,-0.010174,0.567370,0.461620,NaN,0.347752,0.466730,0.036732,0.001170,0.010320,0.024112
bedroomcnt,0.006501,-0.268979,-0.163225,0.001866,0.011559,0.635158,1.000000,0.109296,0.618096,0.020352,0.630772,0.661127,0.035246,0.135834,0.612686,0.143530,0.258134,0.120035,0.070000,-0.008089,-0.110508,0.100248,0.090223,0.044553,0.113805,0.069047,0.035937,-0.021982,-0.011937,0.015233,0.167181,0.102137,0.048581,-0.023416,0.296931,0.239077,NaN,0.178458,0.246231,0.013485,0.007817,0.007744,0.027974
buildingqualitytypeid,-0.047052,0.307971,-0.627714,0.005646,NaN,0.524698,0.109296,1.000000,0.524056,NaN,0.413596,0.492999,NaN,NaN,0.524056,-0.006604,NaN,-0.006604,0.144524,-0.203304,0.113424,0.272421,NaN,-0.006604,NaN,0.272103,0.061744,-0.079485,NaN,0.017317,NaN,-0.195047,0.570913,NaN,0.453166,0.328179,NaN,0.234491,0.328715,0.051050,-0.001301,0.024324,-0.012583
calculatedbathnbr,0.014576,-0.122148,-0.316749,0.003110,0.026579,1.000000,0.618096,0.524056,1.000000,0.049272,0.807138,0.810018,0.029302,0.105947,0.984481,0.140230,0.193002,0.167014,0.020721,-0.0